In [1]:
!pip install web3
!pip install bit

# Imports

In [2]:
import subprocess
import json
import os
from dotenv import load_dotenv
from constants import *

In [3]:
from web3 import Web3
from eth_account import Account

In [35]:
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI

In [5]:
load_dotenv()

True

# Functions

In [6]:
def derive_wallets(mnemonic, coin_list):
    
    wallets = {}

    for i in range(len(coin_list)):        
        
        # Get current coin
        coin = coin_list[i]        
       
        # Build command args
        cmd = 'php derive -g --mnemonic="' + mnemonic + '" --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --coin=' + coin + ' --numderive=10 --format=json'
    
        # Connect to hd-wallet process, pipe results
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
        output, err = p.communicate()
        p_status = p.wait()

        # Get keys in json
        keys = json.loads(output)
        
        # Add coind specific wallets
        wallets[coin] = keys

    return wallets

In [7]:
def priv_key_to_account(coin, priv_key):
    
    if (coin == ETH):
        return Account.privateKeyToAccount(priv_key)
    elif (coin == BTCTEST):
        return PrivateKeyTestnet(priv_key)
    
    return None

In [31]:
def create_tx(coin, account, to, amount):
    
    if (coin == ETH):
        
        gasEstimate = w3.eth.estimateGas({ "from": account.address, "to": to, "value": amount})
            
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address) 
        }

    elif (coin == BTCTEST):
        
        return PrivateKeyTestnet.prepare_transaction(
            account.address, 
            [(to, amount, BTC)])
    
    return None

In [37]:
def send_tx(coin, account, to, amount):
    
    raw_tx = create_tx(coin, account, to, amount)
    signed_tx = signed_tx = account.sign_transaction(raw_tx)
    
    if (coin == ETH):  
        w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        
    elif (coin == BTCTEST):
        NetworkAPI.broadcast_tx_testnet(signed_tx)
    
    return signed_tx

# Configuration

In [10]:
# Get mnemonic from env vars but use default value if not found
mnemonic = os.getenv('MNEMONIC_HW19', 'improve lion original smoke tail reveal boy ability weird lawsuit immune quality')

In [41]:
# Init w3
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [12]:
# Init coin list
coin_list = [BTC, BTCTEST, ETH]

In [13]:
# Init coins dictionary for fast lookup
coins = derive_wallets(mnemonic, coin_list)

In [15]:
# Bitcoin TestNet addresses
btc_test_from = coins[BTCTEST][0]['address']
btc_test_to = coins[BTCTEST][1]['address']

print(f'*** Bitcoin TestNet ***')
print(f'- from address: {btc_test_from}')
print(f'- to address: {btc_test_to}')

*** Bitcoin TestNet ***
- from address: mtUYCvJjnb5gCf1ifC3zTFpf2gCSSP9aBH
- to address: mhRfsyWnTzcSUzUbCcV4UrAiWVSc1P9skM


In [16]:
# Ethereum addresses
eth_from = coins[ETH][0]['address']
eth_to = coins[ETH][1]['address']

print(f'*** Ethereum Test ***')
print(f'- from address: {eth_from}')
print(f'- to address: {eth_to}')

*** Ethereum Test ***
- from address: 0xcBc170C299E841dB273c23262570B63DD5Fe5531
- to address: 0x00Cd84679Fab56696Baf12314Bfc4F9fBd946EC8


# Bitcoin TestNet Transactions

In [17]:
# Init acct for add1
btc_test_act = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
btc_test_act

<PrivateKeyTestnet: mtUYCvJjnb5gCf1ifC3zTFpf2gCSSP9aBH>

In [39]:
# Send .01 BTC from add1 to add2
result = send_tx(BTCTEST, btc_test_act, btc_test_to, .01)

# Output signed transaction
result

'01000000018f138fa04f0bbe5ff8eb93b137bfe369ff5f5d8d506292fe61d4263a6050ad6f010000006a473044022062d52144254654fd6101e11504547bfec242a26e458f3194758d10821e44998e02206f9493cd626c825f008e49a3e54e3d0312de1e3fd17e5dc1271008f2658d0b0701210314dfd94b0fa01bd8f7a37b9a43bbf5c1fddfa4e1b37298df65bced4fdebe9dd9ffffffff0240420f00000000001976a91414f07815989b9a2bbaeb6e817defe1ae6e100a2f88ac52e0d001000000001976a9148e24b1f7723e9923590453dfd6a02a100b8287f188ac00000000'

# Ethereum Test Transactions

In [46]:
# Init acct for add1
eth_act = priv_key_to_account(ETH, coins[ETH][0]['privkey'])
eth_act

In [51]:
# Send 5 ETH from add1 to add2
eth_result = send_tx(ETH, eth_act, eth_to, 5)

# Output signed transaction
eth_result

SignedTransaction(rawTransaction=HexBytes('0xf864808504a817c8008252089400cd84679fab56696baf12314bfc4f9fbd946ec805801ba0641b65d4d78f3142dd1afe337ceaa674c7c9f3e050afaeb136585967f65fca6fa057cc32b64bfdbece0533e8081bbbadbdd3328df1b67b5ec5785d944ab9a1a965'), hash=HexBytes('0x5997dde8695732733f21550853d4d5a87a7116181806d681a9ac45e57ee4629d'), r=45279692543666423153447733559079796413807271053891692071877935643416323934831, s=39712004629989405791151302262367807160848092398002048164199108688998530394469, v=27)

# Testing

In [ ]:
# signed_tx = signed_tx = account.sign_transaction(raw_tx)

In [ ]:
# raw_tx = create_tx(BTCTEST, btc_test_act, btc_test_to, .01)

In [ ]:
# signed_tx = account.sign_transaction(raw_tx)

In [ ]:
# NetworkAPI.broadcast_tx_testnet(signed)

In [ ]:
# w3.eth.getBalance(coins[ETH][0]['privkey'])